In [3]:
from requests import get
API_URL = 'https://api.zotero.org/groups/5766383/items'
json = get(API_URL, params={'limit': '100', 'start': '0'}).json()

json

[{'key': 'CKAI82V9',
  'version': 101,
  'library': {'type': 'group',
   'id': 5766383,
   'name': 'g2c',
   'links': {'alternate': {'href': 'https://www.zotero.org/groups/g2c',
     'type': 'text/html'}}},
  'links': {'self': {'href': 'https://api.zotero.org/groups/5766383/items/CKAI82V9',
    'type': 'application/json'},
   'alternate': {'href': 'https://www.zotero.org/groups/g2c/items/CKAI82V9',
    'type': 'text/html'}},
  'meta': {'createdByUser': {'id': 15763273,
    'username': 'owenleonard11',
    'name': '',
    'links': {'alternate': {'href': 'https://www.zotero.org/owenleonard11',
      'type': 'text/html'}}},
   'creatorSummary': 'Velkova',
   'parsedDate': '2024-02-16',
   'numChildren': 0},
  'data': {'key': 'CKAI82V9',
   'version': 101,
   'itemType': 'webpage',
   'title': 'Dismantling public values, one data center at the time',
   'creators': [{'creatorType': 'author',
     'firstName': 'Julia',
     'lastName': 'Velkova'}],
   'abstractNote': 'Are the Nordic states 

In [4]:
citemap: dict[str, str] = {}

for item in json:
    meta, data = item['meta'], item['data']
    citekey_first = meta.get('creatorSummary', 0) or ' '.join(data['title'].split(' ')[0:2])
    citekey_second = meta.get('parsedDate', '')[0:4]
    citekey_no, citekey = 0, f'{citekey_first} {citekey_second}'
    while citekey in citemap:
        citekey_no += 1
        citekey = f'{citekey_first} {citekey_second} ({citekey_no})'
    citemap[citekey] = data['key']

len(citemap)

94

In [13]:
CITATION_OPTIONS = {
    'format': 'json',
    'include': 'citation',
    'style': 'chicago-fullnote-bibliography'
}

bib: dict[str, str] = {}
for citekey, key in citemap.items():
    response = get(f'{API_URL}/{key}', CITATION_OPTIONS)
    response.raise_for_status()
    if response.status_code != 204:
        bib[citekey] = response.json()['citation']
    else:
        print(response)

bib

{'Velkova 2024': '<span>Julia Velkova, “Dismantling Public Values, One Data Center at the Time,” Linköping University, February 16, 2024, https://liu.se/en/research/megabytes-vs-megawatts/dismantling-public-values-one-data-center-at-the-time.</span>',
 'Taylor 2018': '<span>Astra Taylor, “The Automation Charade,” Logic(s) Magazine, August 1, 2018, https://logicmag.io/failure/the-automation-charade/.</span>',
 'Taylor 2019': '<span>A.R.E. Taylor, “The Data Center as Technological Wilderness,” <i>Culture Machine</i>, The Nature of Data Centers, 18 (2019).</span>',
 'Schmidt 2022': '<span>Florian A. Schmidt, “The Planetary Stacking Order of Multilayered Crowd-AI Systems,” in <i>Digital Work in the Planetary Market</i>, ed. Mark Graham and Fabian Ferrari (The MIT Press, 2022), 137–56, https://doi.org/10.7551/mitpress/13835.003.0012.</span>',
 'Roberts 2021': '<span>Sarah T. Roberts, “Your AI Is a Human,” in <i>Your Computer Is on Fire</i>, ed. Thomas S. Mullaney et al. (The MIT Press, 2021

In [14]:
from json import dump

with open('src/lib/bib.json', 'w', encoding='utf-8') as fp:
    dump(bib, fp, ensure_ascii=False, indent=4)